In [1]:
#profitableHospital-project

In [2]:
#dependencies
import pandas as pd
import requests
import json
import scipy.stats as st
from scipy.stats import linregress
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import tensorflow
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import joblib

In [3]:
# ... File Imports

In [4]:
#2017 Hospital Data from Centers for Medicare Services
df = pd.read_csv("Medicare_Hospital_Cost_Report_PUF_2017.csv")
df.head(2)

C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (105) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Provider CCN,Hospital Name,Street Address,City,State Code,Zip Code,County,Medicare CBSA Number,Rural Versus Urban,CCN Facility Type,...,Total Other Income,Total Income,Total Other Expenses,Net Income,Cost To Charge Ratio,Net Revenue from Medicaid,Medicaid Charges,Net Revenue from Stand-Alone SCHIP,Stand-Alone SCHIP Charges,rpt_rec_num
0,510013,REYNOLDS MEMORIAL HOSPITAL,800 WHEELING AVENUE,GLEN DALE,WV,26038,MARSHALL,48540.0,U,STH,...,4492364.0,2638799.0,NaN,2638799.0,0.453498,1237912.0,4026663.0,NaN,NaN,599181
1,340106,SANDHILLS REGIONAL MEDICAL CENTER,1000 WEST HAMLET AVE,HAMLET,NC,28345,RICHMOND,99934.0,R,STH,...,31174.0,-1114660.0,NaN,-1114660.0,0.173114,391215.0,4981119.0,NaN,NaN,601095


In [5]:
#Quick Check
df['Provider CCN'].nunique()

6075

In [6]:
#Quick Check
df['Provider CCN'].count()

6171

In [7]:
df.drop_duplicates(inplace = True)
df['Provider CCN'].count()

6171

In [8]:
#Certificate of Need Status by State File
conTable = pd.read_csv("certificateofneedstatesTable.csv")
conTable.head(2)

,State Code,CON
0,AK,1
1,AL,1


In [9]:
#String to Numeric Conversion Table for Urban v. Rural designation
urbanTable = pd.read_csv("ruralversusurbanTable.csv")
urbanTable.head()

,Rural Versus Urban,Urban
0,U,1
1,R,0


In [10]:
#Lattitude & Longitude for the 2017 hospital listing
geoTable = pd.read_csv("ccn2geocode.csv")
geoTable.head(2)

,Provider CCN,lat,lng
0,10001,31.215884,-85.363008
1,10005,34.222103,-86.160908


In [11]:
#Quick Check
geoTable['Provider CCN'].count()

6075

In [12]:
# ... Appending of source files into one main working file

In [13]:
mainFile1 = df

In [14]:
#Append lattitude & longitude
mainFile1 = pd.merge(mainFile1, geoTable, how= "left", on=['Provider CCN'])
mainFile1.head(2)

,Provider CCN,Hospital Name,Street Address,City,State Code,Zip Code,County,Medicare CBSA Number,Rural Versus Urban,CCN Facility Type,...,Total Other Expenses,Net Income,Cost To Charge Ratio,Net Revenue from Medicaid,Medicaid Charges,Net Revenue from Stand-Alone SCHIP,Stand-Alone SCHIP Charges,rpt_rec_num,lat,lng
0,510013,REYNOLDS MEMORIAL HOSPITAL,800 WHEELING AVENUE,GLEN DALE,WV,26038,MARSHALL,48540.0,U,STH,...,NaN,2638799.0,0.453498,1237912.0,4026663.0,NaN,NaN,599181,39.946236,-80.752966
1,340106,SANDHILLS REGIONAL MEDICAL CENTER,1000 WEST HAMLET AVE,HAMLET,NC,28345,RICHMOND,99934.0,R,STH,...,NaN,-1114660.0,0.173114,391215.0,4981119.0,NaN,NaN,601095,34.901612,-79.708561


In [15]:
#Quick Check
mainFile1['Provider CCN'].count()

6171

In [16]:
#Append certificate of need indicator
mainFile1 = pd.merge(mainFile1, conTable, how= "left", on=['State Code'])

In [17]:
#Quick Check
mainFile1['Provider CCN'].count()

6171

In [18]:
#Append a numeric representation of rural v. urban
mainFile1 = pd.merge(mainFile1, urbanTable, how= "left", on=['Rural Versus Urban'])

In [19]:
#Quick Check
mainFile1['Provider CCN'].count()

6171

In [20]:
mainFile1.head(2)

,Provider CCN,Hospital Name,Street Address,City,State Code,Zip Code,County,Medicare CBSA Number,Rural Versus Urban,CCN Facility Type,...,Cost To Charge Ratio,Net Revenue from Medicaid,Medicaid Charges,Net Revenue from Stand-Alone SCHIP,Stand-Alone SCHIP Charges,rpt_rec_num,lat,lng,CON,Urban
0,510013,REYNOLDS MEMORIAL HOSPITAL,800 WHEELING AVENUE,GLEN DALE,WV,26038,MARSHALL,48540.0,U,STH,...,0.453498,1237912.0,4026663.0,NaN,NaN,599181,39.946236,-80.752966,1.0,1.0
1,340106,SANDHILLS REGIONAL MEDICAL CENTER,1000 WEST HAMLET AVE,HAMLET,NC,28345,RICHMOND,99934.0,R,STH,...,0.173114,391215.0,4981119.0,NaN,NaN,601095,34.901612,-79.708561,1.0,0.0


In [21]:
#mainFile1.to_csv("main1_2017.csv", index=False, header=True)

In [22]:
# ... Add calculated fields

In [23]:
mainFile2 = mainFile1

In [24]:
#labor Expense
lbrexpense = mainFile2["Wage-Related Costs (Core)"] + mainFile2["Total Salaries (adjusted)"]
mainFile2["Labor Exp"] = lbrexpense
#wagepercent.head()

In [25]:
#Append calculated field inpatient percent
inpatient = mainFile2["Inpatient Revenue"]/mainFile2["Gross Revenue"]
mainFile2["Inpatient %"] = inpatient

In [26]:
#Append calculated field outpatient percent
outpatient = mainFile2["Outpatient Revenue"]/mainFile2["Gross Revenue"]
mainFile2["Outpatient %"] = outpatient

In [27]:
#Append calculated field charity percent
charity = mainFile2["Cost of Charity Care"]/mainFile2["Net Patient Revenue"]
mainFile2["Charity %"] = charity

In [28]:
#Append calculated field uncompensated percent
uncompensated = mainFile2["Cost of Uncompensated Care"]/mainFile2["Net Patient Revenue"]
mainFile2["Uncompensated %"] = charity

In [29]:
#Append calculated field revenue per discharge
revdischarge = mainFile2["Net Patient Revenue"]/mainFile2["Total Discharges (V + XVIII + XIX + Unknown)"]
mainFile2["Rev/Discharge"] = revdischarge

In [30]:
#Append calculated field revenue per bed
revbed = mainFile2["Net Patient Revenue"]/mainFile2["Number of Beds"]
mainFile2["Rev Per Bed"] = revbed

In [31]:
#Append calculated field revenue per bed day
revbed = mainFile2["Net Patient Revenue"]/mainFile2["Total Bed Days Available"]
mainFile2["Rev Per Bed Days"] = revbed

In [32]:
#Append calculated field revs per fte
revfte = mainFile2["Net Patient Revenue"]/mainFile2["FTE - Employees on Payroll"]
mainFile2["Rev Per FTE"] = revfte

In [33]:
#Append calculated field discharges per bed
volbed = mainFile2["Total Discharges (V + XVIII + XIX + Unknown)"]/mainFile2["Number of Beds"]
mainFile2["Discharges Per Bed"] = volbed

In [34]:
#Append calculated field discharges per bed day
voldays = mainFile2["Total Discharges (V + XVIII + XIX + Unknown)"]/mainFile2["Total Bed Days Available"]
mainFile2["Discharges Per Bed Days"] = voldays

In [35]:
#Append calculated field discharges per bed
volfte = mainFile2["Total Discharges (V + XVIII + XIX + Unknown)"]/mainFile2["FTE - Employees on Payroll"]
mainFile2["Discharge/FTE"] = volbed

In [36]:
#labor percent
wagepercent = mainFile2["Wage-Related Costs (Core)"] + mainFile2["Total Salaries (adjusted)"]
wagepercent = wagepercent / mainFile2["Net Patient Revenue"]
mainFile2["Labor %"] = wagepercent

In [37]:
#Overhead percent
overheadpercent = mainFile2["Overhead Non-Salary Costs"] / mainFile2["Net Patient Revenue"]
mainFile2["Overhead %"] = overheadpercent

In [38]:
#Depreciation percent
deprpercent = mainFile2["Depreciation Cost"] / mainFile2["Net Patient Revenue"]
mainFile2["Depreciation %"] = deprpercent

In [39]:
#labor Per FTE
wagefte = mainFile2["Wage-Related Costs (Core)"] + mainFile2["Total Salaries (adjusted)"]
wagefte = wagefte / mainFile2["FTE - Employees on Payroll"]
mainFile2["LbrExp/FTE"] = wagefte

In [40]:
#Overhead Expense Per Bed
overbed = mainFile2["Overhead Non-Salary Costs"] / mainFile2["Number of Beds"]
mainFile2["Overhead Exp Per Bed"] = overbed

In [41]:
#Append calculated field net income percent to file
profit = mainFile2["Net Income"]/mainFile2["Net Patient Revenue"]
mainFile2["Net Income %"] = profit

In [42]:
# Review net income percent for outlier reduction approach
netinc = mainFile2["Net Income %"]

In [43]:
# Creating variable for quartile work
netinc2 = netinc.dropna(how='any')
chartdata = netinc2

In [44]:
#Quartile calculations
quartiles = chartdata.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile of profit percent is: {lowerq}")
print(f"The upper quartile of profit percent is: {upperq}")
print(f"The interquartile range of profit percent is: {iqr}")
print(f"The the median of profit percent is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

The lower quartile of profit percent is: -0.030653238847723918
The upper quartile of profit percent is: 0.11379730445563284
The interquartile range of profit percent is: 0.14445054330335677
The the median of profit percent is: 0.0380451602645128 
Values below -0.24732905380275907 could be outliers.
Values above 0.330473119410668 could be outliers.


In [45]:
#Create a bronze, silver, gold score for clasifying net income performance
bins = [-100, -0.030605, 0.113797, 100]
group_names = ["3", "2", "1"]

In [46]:
#Add a bronze, silver, gold scoring system to the dataset to support machine learning
mainFile2["Net Income Score"] = pd.cut(mainFile2["Net Income %"], bins, labels=group_names, include_lowest=True)
#mainFile2.head()

In [47]:
#Quick check on distribution of bronze, silver and gold
mainFile2['Net Income Score'].value_counts()

2    2945
1    1473
3    1473
Name: Net Income Score, dtype: int64

In [48]:
#Pacesetter status
bins2 = [-100, -0.030605, .038045,  0.11379, 100]
group_names2 = ["4th Quartile", "3rd Quartile", "2nd Quartile", "1st Quartile"]

In [49]:
#Add Pacesetter Status to dataframe
mainFile2["Pacesetter Group"] = pd.cut(mainFile2["Net Income %"], bins2, labels=group_names2, include_lowest=True)
#mainFile2.head()

In [50]:
mainFile2['Pacesetter Group'].value_counts()

1st Quartile    1473
2nd Quartile    1473
4th Quartile    1473
3rd Quartile    1472
Name: Pacesetter Group, dtype: int64

In [51]:
#Quick Check
mainFile2['Provider CCN'].count()

6171

In [52]:
#mainFile2.to_csv("main2_2017.csv", index=False, header=True)

In [53]:
# ..... File targeting

In [54]:
mainFile3 = mainFile2
mainFile3['Provider CCN'].count()

6171

In [55]:
mainFile3['lat'].dtype

dtype('float64')

In [56]:
#Remove the Sitka exceptions (addresses that did not match when called in API)
mainFile3 = mainFile3.loc[mainFile3['lat'] <= 48.85]

In [57]:
#Removes state
mainFile3 = mainFile3.loc[mainFile3['State Code'] != 'PR']

In [58]:
#Removes state
mainFile3 = mainFile3.loc[mainFile3['State Code'] != 'GU']

In [59]:
#Removes state
mainFile3 = mainFile3.loc[mainFile3['State Code'] != 'VI']

In [60]:
#Removes state
mainFile3 = mainFile3.loc[mainFile3['State Code'] != 'MP']

In [61]:
#Removes state
mainFile3 = mainFile3.loc[mainFile3['State Code'] != 'HI']

In [62]:
#Removes state
mainFile3 = mainFile3.loc[mainFile3['State Code'] != 'AK']

In [63]:
#Quick Check ... Lose 117 to state code reduction
mainFile3['Provider CCN'].count()

6026

In [64]:
#Remove hospitals that did not report revenue (Lose 273 to missing or unreported revenue events)
mainFile3 = mainFile3.loc[mainFile3['Net Patient Revenue'] >= 0.1]

In [65]:
#Quick Check
mainFile3['Provider CCN'].count()

5756

In [66]:
#Remove hospitals that discharged less than 15 patients (Lose 49 by placing a 15 patient minimum for study eligibility)
mainFile3 = mainFile3.loc[mainFile3['Total Discharges (V + XVIII + XIX + Unknown)'] >= 15]

In [67]:
#Quick Check
mainFile3['Provider CCN'].count()

5708

In [68]:
#Remove hospitals with less than 10 employees on payroll (Lose 23 by placing a 10 employee minimum for study eligibility)
mainFile3 = mainFile3.loc[mainFile3['FTE - Employees on Payroll'] >= 10]

In [69]:
#Quick Check
mainFile3['Provider CCN'].count()

5685

In [70]:
#Targets dataset on STH locations (~56% of eligible continental US locations are STH providers)
mainFile3 = mainFile3.loc[mainFile3['CCN Facility Type'] == 'STH']

In [71]:
#Quick Check
mainFile3['Provider CCN'].count()

3184

In [72]:
#Quick Check
mainFile3['Provider CCN'].count()

3184

In [73]:
#mainFile3.to_csv("main3_2017.csv", index=False, header=True)

In [74]:
# ... Target file based on relevant features

In [75]:
mainFile4 = mainFile3

In [76]:
#Targeting the dataset on metrics based on Kent's judgement
mainFile4 = mainFile4[['Provider CCN','Hospital Name','Street Address','City','State Code','Zip Code', 'lat', 'lng', 'Rural Versus Urban', 'CCN Facility Type', 'Fiscal Year Begin Date', 'Fiscal Year End Date', 'FTE - Employees on Payroll', 'Number of Beds', 'Total Bed Days Available', 'Total Discharges (V + XVIII + XIX + Unknown)', 'Net Patient Revenue', 'Net Income', 'Wage-Related Costs (Core)', 'Total Salaries (adjusted)', 'Contract Labor', 'Rev Per FTE', 'CON','Urban','Inpatient %','Outpatient %','Charity %','Uncompensated %','Rev/Discharge','Rev Per Bed','Rev Per Bed Days', 'Discharges Per Bed','Discharges Per Bed Days','Discharge/FTE','Net Income %', 'Net Income Score', 'Labor %', 'Overhead %', 'Depreciation %', 'LbrExp/FTE', 'Overhead Exp Per Bed', 'Pacesetter Group']]
mainFile4.head(2)


,Provider CCN,Hospital Name,Street Address,City,State Code,Zip Code,lat,lng,Rural Versus Urban,CCN Facility Type,...,Discharges Per Bed Days,Discharge/FTE,Net Income %,Net Income Score,Labor %,Overhead %,Depreciation %,LbrExp/FTE,Overhead Exp Per Bed,Pacesetter Group
0,510013,REYNOLDS MEMORIAL HOSPITAL,800 WHEELING AVENUE,GLEN DALE,WV,26038,39.946236,-80.752966,U,STH,...,0.067547,6.214286,0.308395,1,0.715116,0.627406,0.031451,18996.972990,76691.942857,1st Quartile
1,340106,SANDHILLS REGIONAL MEDICAL CENTER,1000 WEST HAMLET AVE,HAMLET,NC,28345,34.901612,-79.708561,R,STH,...,0.050205,3.062500,-0.337043,3,0.509928,0.924882,0.073496,61909.838473,47792.984375,4th Quartile


In [77]:
# Identify incomplete rows
mainFile4.count()

Provider CCN                                    3184
Hospital Name                                   3184
Street Address                                  3184
City                                            3184
State Code                                      3184
Zip Code                                        3184
lat                                             3184
lng                                             3184
Rural Versus Urban                              3184
CCN Facility Type                               3184
Fiscal Year Begin Date                          3184
Fiscal Year End Date                            3184
FTE - Employees on Payroll                      3184
Number of Beds                                  3184
Total Bed Days Available                        3184
Total Discharges (V + XVIII + XIX + Unknown)    3184
Net Patient Revenue                             3184
Net Income                                      3183
Wage-Related Costs (Core)                     

In [78]:
mainFile4 = mainFile4.dropna(how='any')

In [79]:
# Identify incomplete rows
mainFile4.count()

Provider CCN                                    2705
Hospital Name                                   2705
Street Address                                  2705
City                                            2705
State Code                                      2705
Zip Code                                        2705
lat                                             2705
lng                                             2705
Rural Versus Urban                              2705
CCN Facility Type                               2705
Fiscal Year Begin Date                          2705
Fiscal Year End Date                            2705
FTE - Employees on Payroll                      2705
Number of Beds                                  2705
Total Bed Days Available                        2705
Total Discharges (V + XVIII + XIX + Unknown)    2705
Net Patient Revenue                             2705
Net Income                                      2705
Wage-Related Costs (Core)                     

In [80]:
# Create a file for machine learning that is not impacted by formatting
mainFile5 = mainFile4

In [81]:
#%'s'
# Machine learning will not run with this format
# chg1percent = mainFile4["Net Income %"]
# chg1percent = chg1percent *100
# mainFile4["Net Income %"] = chg1percent
# mainFile4["Net Income %"] = mainFile4["Net Income %"].map("{:.4f}%".format)

# chg2percent = mainFile4["Labor %"]
# chg2percent = chg2percent *100
# mainFile4["Labor %"] = chg2percent
# mainFile4["Labor %"] = mainFile4["Labor %"].map("{:.4f}%".format)

# chg3percent = mainFile4["Overhead %"]
# chg3percent = chg3percent *100
# mainFile4["Overhead %"] = chg3percent
# mainFile4["Overhead %"] = mainFile4["Overhead %"].map("{:.4f}%".format)

# chg4percent = mainFile4["Depreciation %"]
# chg4percent = chg4percent *100
# mainFile4["Depreciation %"] = chg4percent
# mainFile4["Depreciation %"] = mainFile4["Depreciation %"].map("{:.4f}%".format)

# chg5percent = mainFile4["Charity %"]
# chg5percent = chg5percent *100
# mainFile4["Charity %"] = chg5percent
# mainFile4["Charity %"] = mainFile4["Charity %"].map("{:.4f}%".format)

# chg6percent = mainFile4["Uncompensated %"]
# chg6percent = chg6percent *100
# mainFile4["Uncompensated %"] = chg6percent
# mainFile4["Uncompensated %"] = mainFile4["Uncompensated %"].map("{:.4f}%".format)

# chg7percent = mainFile4["Inpatient %"]
# chg7percent = chg7percent *100
# mainFile4["Inpatient %"] = chg7percent
# mainFile4["Inpatient %"] = mainFile4["Inpatient %"].map("{:.4f}%".format)

# chg8percent = mainFile4["Outpatient %"]
# chg8percent = chg8percent *100
# mainFile4["Outpatient %"] = chg8percent
# mainFile4["Outpatient %"] = mainFile4["Outpatient %"].map("{:.4f}%".format)

# #$'s'
# mainFile4["Net Patient Revenue"] = mainFile4["Net Patient Revenue"].map("${:,.0f}".format)
# mainFile4["Net Income"] = mainFile4["Net Income"].map("${:.0f}".format)
# mainFile4["Wage-Related Costs (Core)"] = mainFile4["Wage-Related Costs (Core)"].map("${:,.0f}".format)
# mainFile4["Total Salaries (adjusted)"] = mainFile4["Total Salaries (adjusted)"].map("${:,.0f}".format)
# mainFile4["Contract Labor"] = mainFile4["Contract Labor"].map("${:,.0f}".format)
# mainFile4["Rev Per FTE"] = mainFile4["Rev Per FTE"].map("${:,.0f}".format)
# mainFile4["Rev/Discharge"] = mainFile4["Rev/Discharge"].map("${:,.0f}".format)
# mainFile4["Rev Per Bed"] = mainFile4["Rev Per Bed"].map("${:,.0f}".format)
# mainFile4["Rev Per Bed Days"] = mainFile4["Rev Per Bed Days"].map("${:,.0f}".format)
# mainFile4["LbrExp/FTE"] = mainFile4["LbrExp/FTE"].map("${:,.0f}".format)
# mainFile4["Overhead Exp Per Bed"] = mainFile4["Overhead Exp Per Bed"].map("${:,.0f}".format)

# ##'s'
# mainFile4["Discharges Per Bed"] = mainFile4["Discharges Per Bed"].map("{:,.2f}".format)
# mainFile4["Discharges Per Bed Days"] = mainFile4["Discharges Per Bed Days"].map("{:,.2f}".format)
# mainFile4["Discharge/FTE"] = mainFile4["Discharge/FTE"].map("{:,.2f}".format)

In [82]:
#Output targeted file
#mainFile4.to_csv("main4_2017.csv", index=False, header=True)

In [158]:
# ... Machine Learning Section 

In [159]:
# Select features to be used as x values
# features1 = mainFile5[['Rev/Discharge','Rev Per Bed','Rev Per Bed Days', 'Discharges Per Bed', 'Discharge/FTE', 'Labor %', 'Overhead %', 'Depreciation %']]
# features1.head(2)
#RFE logrithmic output ... array([7(r/d), 6(r/b), 5(r/bday), 8(d/b), 3(d/fte), 2(l%), 1(o%, 4(d%)])

In [160]:
# Select features to be used as x values
features1 = mainFile5[['Rev/Discharge','Rev Per Bed','Rev Per Bed Days', 'Discharge/FTE', 'Labor %', 'Overhead %', 'Depreciation %']]
features1.head(2)
#RFE logrithmic output ... array([7(r/d), 6(r/b), 5(r/bday), 8(d/b), 3(d/fte), 2(l%), 1(o%), 4(d%)])

,Rev/Discharge,Rev Per Bed,Rev Per Bed Days,Discharge/FTE,Labor %,Overhead %,Depreciation %
1,16873.362245,51674.671875,847.125768,3.062500,0.509928,0.924882,0.073496
5,15444.529412,132660.378947,1105.503158,8.589474,0.414191,0.826683,0.119266


In [161]:
#Output features file as a reference document
features1.to_csv("rfefocusedfeatures2017.csv", index=False, header=True)

In [162]:
# ... Linear Regression Model

In [163]:
# Identify the outcome column/data
y = mainFile5[['Net Income %']]
y.head()

,Net Income %
1,-0.337043
5,-0.132426
6,-0.291462
7,-0.050419
8,0.080314


In [164]:
mainFile5["Net Income %"].dtype

dtype('float64')

In [165]:
y.shape

(2705, 1)

In [166]:
X = features1
X.shape

(2705, 7)

In [167]:
#Create a training and testing dataset - one hot
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [168]:
y_train.shape

(2028, 1)

In [169]:
X_train.shape

(2028, 7)

In [170]:
y_test.shape

(677, 1)

In [171]:
X_test.shape

(677, 7)

In [172]:
# Scale your data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [173]:
#Create the linear regression model and fit to the training data
#score is r2 root mean ... 1 is better
modela = LinearRegression()
modela.fit(X_train_scaled, y_train)
score = modela.score(X_train_scaled, y_train)
print(f"Training Data Score: {modela.score(X_train_scaled, y_train)}")

Training Data Score: 0.42847526907979117


In [174]:
#Run the model with the testing data and determine score
print(f"Testing Data Score: {modela.score(X_test_scaled, y_test)}")

Testing Data Score: -0.3507256656524924


In [175]:
# ... Linear Recursive Feature Elimination

In [176]:
#Identify features to create a more efficient model with
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=5, step=1)
selector = selector.fit(X_train_scaled, y_train)

In [177]:
selector.support_

array([ True,  True,  True, False,  True,  True, False])

In [178]:
selector.ranking_

array([1, 1, 1, 2, 1, 1, 3])

In [179]:
# ... Logistics Model

In [180]:
# Identify the outcome column/data
yy = mainFile5[['Net Income Score']]
yy.head()

,Net Income Score
1,3
5,3
6,3
7,3
8,2


In [181]:
yy.shape

(2705, 1)

In [182]:
XX = features1
XX.shape

(2705, 7)

In [183]:
#Create a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(
    XX, yy, random_state=1)

In [184]:
y_train.shape

(2028, 1)

In [185]:
X_train.shape

(2028, 7)

In [186]:
y_test.shape

(677, 1)

In [187]:
y_test.shape

(677, 1)

In [188]:
# Scale your data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [189]:
#Create the logistic regression model and fit to the training data
#score is r2 root mean ... 1 is better
modela = LogisticRegression()
modela.fit(X_train_scaled, y_train)
score = modela.score(X_train_scaled, y_train)
print(f"Training Data Score: {modela.score(X_train_scaled, y_train)}")

Training Data Score: 0.5956607495069034


C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [190]:
#Run the model with the testing data and determine score
print(f"Testing Data Score: {modela.score(X_test_scaled, y_test)}")

Testing Data Score: 0.6233382570162481


In [191]:
# ... recursive feature elimination

In [192]:
#Identify features to create a more efficient model with
estimator = LogisticRegression()
selector = RFE(estimator, n_features_to_select=1, step=1)
selector = selector.fit(X_train_scaled, y_train)

C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Pl

In [193]:
selector.support_

array([False, False, False, False, False,  True, False])

In [194]:
selector.ranking_

array([7, 6, 5, 3, 2, 1, 4])

In [119]:
#End ... kent matthew rodgers, profitableHospital-project

In [ ]:
# Useful scraps

In [ ]:
# geoTable.drop_duplicates(inplace = True)
# geoTable['Provider CCN'].count()
#df.count()
#mainFile1['Provider CCN'].duplicated()